# 一、语言与语法

1. 首先需要构建一个简单的便于形成迭代模型的语言定义
1. 而后需要构建语言转换规则，使得可以应用于成为优化调节
1. 不进行解题，而是只用于优化，即问题的化解

# 二、以排序为例

## 1. 输入
Input: x[5]

## 2. 处理
### 做或是求解 ，先考虑Do，优化的Do，而不考虑反解问题
Do or Slove: 

``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_ind_If=if(x_1<x_2)
  x_val_If=if(x[x_2]<x[x_1])
  x_If=x_ind_If & x_val_If
  swap(x_If)(x,x_i,x_k)
``` 

## 3. 输出
Output: x[5]

# 三、初步实现与建模

## 1 模型转换
#### 定义过程型
|P4Short|P     |复杂度  |  描述|
|----|----     |---  |----|
|FR  |ForRange |x   |   遍历输入量x产生输出集合0..1..x,作为一个块看待,可输出ind，以及标识结束，作为后续依赖的标志|
|IS  |IfSmall  |1   |   判断是否左x1小于右侧x2|
|GS  |GetSize  |1   | 获得x集合的长度|
|GV  |GetValue |1   | 获得x结合的indx位置的值|
|SV  |SetValue |1   | 设置x结合的indx位置的值|
|LA  |LogicAnd |1   | 逻辑与运算|
|SW  |swap     |1  | 交换x的ind1和ind2位置
|IM  |InsertMap|log(x)    |插入Map数据结构|
|GM  |GetMap|1    |获得Map结构中的一项|

|说明|
|:---|
| 因此依赖关系针对FR分为两种，一种为获得其中一项作为FR的内部依赖，一种为FR的结束依赖|
| FR是函数的抽象，普通函数为FR为1的函数|
|函数本身也决定了一种独特的依赖关系，内部依赖和外部依赖，是否可以将两种依赖统一呢|
|目前两种依赖关系等效于：即非树形，也非完全的AST语法树|

#### 过程输入输出表（图?CNN?）

##### P2，P3输入输出上没有体现出依赖关系，但是实质是存在依赖性的，即P3必须依赖于P2，是否依赖关系要重新建立一张表呢？

|N  |P    |P_pre  |    x|  x_len|   x_1|   x_2| x_1_val |x_2_val | x_ind_If | x_val_If| x_If|
|---|---  |---    |---  |---   |---   |---   |---    |---     |---  | ----|  ----|
|  1| GS  |0     |   I  |    O |
|  2| FR  |1     |&nbsp; |    I |    O|
|  3| FR  |2     |&nbsp; |    I |&nbsp;|    O|
|  4| GV  |3     | I    |&nbsp; | I   |&nbsp;| O|   
|  5| GV  |3     | I    |&nbsp; |&nbsp;|I    | &nbsp; |       O|
|  6| IS  |3     | &nbsp;| &nbsp;|   I |    I| &nbsp; |  &nbsp;  |    O|
|  7| IS  |3     | &nbsp;| &nbsp;|&nbsp;|&nbsp;| I     |  I |   &nbsp;|  O |
|  8| LA  |4,5,6,7 | &nbsp;| &nbsp;|&nbsp;|&nbsp;| &nbsp;  |&nbsp; |   I|  I | O|
|  9| SW  |8     | O    |&nbsp; |I    | I   | &nbsp; | &nbsp;   |&nbsp;|  &nbsp;|I|

#### 依赖关系表
|N  |P1    |P2    |P3    |P4    |P5    |P6    |P7    |P8
|---|---  |---    |---  |---   |---   |---   |---    |--- |
| P1|
| P2|Y|
| P3|&nbsp;|Y|
| P4|&nbsp;|Y|
| P5|&nbsp;|&nbsp;|Y|
| P6|&nbsp;|Y|Y|
| P7|&nbsp;|&nbsp;|&nbsp;|Y|Y|
| P8|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|Y|
| P9|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|


## 2 复杂度计算

``` go
P1 = 1
P2 = P1 * (x_len) 
P3 = P2 * (x_len)
P4 = P3 * (1)
P5 = P3 * (1)
P6 = P2 + P3
P7 = P4 + P5
P8 = P6 + P7
P9 = P8

==>

P9 = P2 + P3 + P4 + P5
  = P1 * (x_len) + P1 * (x_len) * (x_len) + P1 * (x_len) * (x_len) + P1 * (x_len) * (x_len)
  = P1 * (x_len) * (x_len)
  ~= N * (x_len) * (x_len)
```

# 四、手动优化

### 1. 冒泡优化循环

``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x_1):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_val_If=if(x[x_2]<x[x_1])
  swap(x_val_If)(x,x_1,x_2)
``` 

#### 过程结果
| N  | P     |    x|  x_len|   x_1|   x_2| x_1_val |x_2_val | x_If |
| ---| ---   |---  |---   |---   |---   |---    |---     |---  |
|  1 | GS    |   I  |    O |
|  2 | FR    |&nbsp; |    I |    O|
|  3 | FR    |&nbsp; | &nbsp;|    I|    O|
|  4 | GV    | I    |&nbsp; | I   |&nbsp;| O|   
|  5 | GV    | I    |&nbsp; |&nbsp;|I    | &nbsp; |       O|
|  6 | IS    | &nbsp;| &nbsp;|&nbsp;|&nbsp;|     I|       I|    O|
|  7 | SW    | O    |&nbsp; |I    | I   | &nbsp; | &nbsp;   |    I|


#### 依赖关系表
|N  |P1    |P2    |P3    |P4    |P5    |P6    |P7    |
|---|---  |---    |---  |---   |---   |---   |---    |
| P1|
| P2|Y|
| P3|&nbsp;|Y|
| P4|&nbsp;|&nbsp;|Y|
| P5|&nbsp;|&nbsp;|Y|
| P6|&nbsp;|&nbsp;|&nbsp;|Y|Y|
| P7|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|

#### 复杂度


### 2. 冒泡优化Map插入
#####  <u>依赖二维关系表，目前只代表顺序关系，不能完全覆盖包含关系</u>

``` go
for(x_1 :0..1..x.len):
  x_1_val=x[x_1]
  IM(x_1_val)
for(x_2 :0..1..x.len):
  x_1_val=GM(x_2)
  x[x_2]=x_1_val
``` 

#### 过程结果
| N  | P     |    x|  x_len|   x_1|   x_2| x_1_val |Map | x_If |
| ---| ---   |---  |---   |---   |---   |---    |---     |---  |
|  1 | GS    |   I  |    O |
|  2 | FR    |&nbsp; |    I |    O|
|  4 | GV    | I    |&nbsp; | I   |&nbsp;| O|   
|  5 | IM    | &nbsp;  |&nbsp; |&nbsp; |&nbsp;|I    | O |       |
|  6 | FR    |&nbsp; |    I |&nbsp; |    O|
|  7 | GM    | &nbsp;    |&nbsp; |&nbsp; | I|O | I   |    
|  8 | SV    | O     |&nbsp; |&nbsp; |I| &nbsp; | I   |   


#### 依赖关系表
|N  |P1    |P2    |P3    |P4    |P5    |P6    |P7    |
|---|---  |---    |---  |---   |---   |---   |---    |
| P1|
| P2|Y|
| P3|&nbsp;|Y|
| P4|&nbsp;|&nbsp;|I|
| P5|&nbsp;|&nbsp;|&nbsp;|Y|
| P6|&nbsp;|&nbsp;|Y|
| P7|&nbsp;|&nbsp;|&nbsp;|&nbsp;|I|
| P8|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|

等效依赖图如下
``` go
B -> P1 -> P2 -> P3 -> P6 -> E
            |    |
            |    P7 -> P8
            |
            P4 -> P5
```

#### 复杂度
P9 = N * x_len * log(x_len)

# 五、手动优化的模型（梯度分析）

## 1.原始输入简化
``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_ind_If=if(x_1<x_2)
  x_val_If=if(x[x_2]<x[x_1])
  x_If=x_ind_If & x_val_If
  swap(x_If)(x,x_i,x_k)

//-----------------------------------
//P行号(过程名，复杂度)
B -> P1(FR,x_len) -> E
    |
    *P2(FR,x_len)
    |
    *P5(IS,1)，P3(GV,1),P4(GV,1) -> P6(IS,1) -> P7(IA,1) -> P8(SW,1)
``` 


## 2.冒泡输入简化
``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x_1):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_val_If=if(x[x_2]<x[x_1])
  swap(x_val_If)(x,x_1,x_2)

//-----------------------------------
B -> P1(FR,x_len) -> E
    |
    *P2（FR,x_len）
    |
    P3(GV,1),*P4(GV,1) -> *P5(IS,1) -> *P6(SW,1)
``` 

## 3.利用Map输入简化
``` go
for(x_1 :0..1..x.len):
  x_1_val=x[x_1]
  IM(x_1_val)
for(x_2 :0..1..x.len):
  x_1_val=GM(x_2)
  x[x_2]=x_1_val
//-----------------------------------
B -> P1(FR,x_len) -> P4(FR,x_len) -> E
    |          |
    |          P6(GM,log) -> P5(SV,1)
    |
    P2(GV,1) -> P4(IM,log)
``` 

## 3.优化规则的抽象
|id|规则|抽象总结|
|---|----|---|
|1|<b> P5的判定是可以沿着路径向回搜索FR,降低进入的可能性 </b>| 减少循环的范围|
|2|<b> 子循环节P2其实是为了得到最大值，可以通过引入Map进行优化 </b>| 消除层级循环 |

## 3. 优化的步骤（单梯度操作）
|id|操作|
|---|---|
|1| 将判断往上个依赖项提升 |
|2| 将FR，和IF进行合并|
|3| 兄弟步骤进行次序更换|
|4| FR中的兄弟步骤可以直接进行FR的分解或合并|
|5| FR中的依赖步骤引入中间变量进行分解
|6| FR循环获得最值，？？？集合与子集有问题|

# 6. 智能模型

#### 定义规则 --> ( 定义人工智能在语法模型中搜索到优化节形成若干个优化方案 --> 进行权值评判来选择某个方案)  --> 实施方案（确定的语法表调整）

#### 辅助工具：普通语言产生两个操作表，编写解释器可以直接运行操作表，编写解释权可以由操作表反向为语言，或者不制作工具，只证明可行性
